In [1]:
import csv
import os
import pandas as pd
import random
from string import Template
from typing import Any, List, Tuple, Union

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [3]:
#######################################################
# CONSTANTS
#######################################################
LANG_ES = 'es'
LANG_EN = 'en'
NODE_IMPORTANCE = 'node_importance'
NODE_NAME = 'node_name'
QUALITY_MEASURE = 'quality_measure'
RAND = 'rand'
RANK = 'rank'
REASON = 'reason'
TARGET = 'target'

In [4]:
class Why:
    
    _LOCAL = 'local'
    _GLOBAL = 'global'
    _SEP_LAST = {
        LANG_ES: ' y ',
        LANG_EN: ' and '
    }
    
    def __init__(self, language: str, local_expl: pd.DataFrame, local_nodes: pd.DataFrame, why_elements: pd.DataFrame,
                 why_target: pd.DataFrame, why_templates: pd.DataFrame, quality_measure: pd.DataFrame,
                 n_local_features: int = 2, n_global_features: int = 2, min_quality: float = 0.0):
        self.language = language
        if self.language not in self._SEP_LAST.keys():
            raise NameError("Language {} not supported".format(language))
        self.local_expl = local_expl
        self.local_nodes = local_nodes
        self.why_elements = why_elements
        self.why_target = why_target
        self.why_templates = why_templates
        self.quality_measure = quality_measure
        self.n_local_features = n_local_features
        self.n_global_features = n_global_features
        self.min_quality = min_quality
        
    def __build_template(self, items: Union[List, Tuple], sep=', ') -> str:
        sep_last = self._SEP_LAST[self.language]
        i_list = [Template.delimiter + i for i in (items if isinstance(items, list) else list(items))]
        return (sep.join(i_list[:-1]) + sep_last + i_list[-1]) if len(i_list) > 1 else i_list[0]
    
    def build_why(self, key_column: str, key_value: Any = None, template_index: Union[str, int] = RAND) -> Union[pd.DataFrame, str]:
        # Check case existence if a single case is requested
        if key_value is None:
            local_expl = self.local_expl
        else:
            local_expl = self.local_expl[self.local_expl[key_column] == key_value]
            if local_expl.shape[0] == 0:
                raise ValueError("Value {} does not exist in column \'{}\'".format(key_value, key_column))
            elif local_expl.shape[0] > 1:
                raise ValueError("More than one row with value {} in column \'{}\'".format(key_value, key_column))
        
        # Build a dataframe with all the case information
        df = (local_expl[[key_column, TARGET]]
              .merge(self.local_nodes[[key_column, NODE_NAME, NODE_IMPORTANCE]], on=key_column, how='inner')
              .merge(self.why_elements.rename(columns={'feature': NODE_NAME}), on=NODE_NAME, how='inner')
              .merge(self.why_target.rename(columns={'feature': NODE_NAME}),
                     on=[TARGET, NODE_NAME], how='inner', suffixes=['_' + self._LOCAL, '_' + self._GLOBAL])
              .merge(self.quality_measure[[key_column, QUALITY_MEASURE]], on=key_column, how='left'))
        df[RANK] = df.groupby(key_column)[NODE_IMPORTANCE].rank(method='dense', ascending=False).astype(int)
        
        max_n_features = max(self.n_local_features, self.n_global_features)
        df_rank = df[df[RANK] <= max_n_features]
        
        def get_single_why(df_single: pd.DataFrame) -> str:
            # Check the quality of the explainability values
            # Specific update for demo: the value of the quality measure must be the complementary
            r = df_single.head(1)
            quality = 1 - r[QUALITY_MEASURE].values[0]
            if quality < self.min_quality:
                return self.why_templates.iloc[0, 0]

            # Build why sentence
            kw_local = dict([('v_' + self._LOCAL + '_' + str(i), v) for i, v in enumerate(df_single['reason_' + self._LOCAL].iloc[:self.n_local_features])])
            kw_global = dict([('v_' + self._GLOBAL + '_' + str(i), v) for i, v in enumerate(df_single['reason_' + self._GLOBAL].iloc[:self.n_global_features])])
            temp_local_explain = self.__build_template(items=list(kw_local))
            temp_global_explain = self.__build_template(items=list(kw_global))

            temp_idx_max = self.why_templates.shape[0] - 1
            temp_idx = random.randint(1, temp_idx_max) if template_index == RAND else min(template_index, temp_idx_max)
            temp_why_str = (Template(Template(self.why_templates.iloc[temp_idx, 0])
                                     .substitute(temp_local_explain=temp_local_explain,
                                                 temp_global_explain=temp_global_explain,
                                                 target=df_single[TARGET].iloc[0]))
                            .substitute(**kw_local, **kw_global)
                            .capitalize())
            return temp_why_str
        
        df_final = df_rank.groupby(key_column).apply(get_single_why).to_frame(REASON).reset_index()
        if key_value is None:
            return df_final
        else:
            return df_final[REASON].values[0]
        

In [5]:
# Path for data within repo
# data_path = './example_why_titanic'

# Path for data generated by examples/exgraph/tef_shap_titanic.py (Alejandro)
# data_path = 'C:/datos/AIP/proyectos/XAIoGraphs/data/why/tmp'

# Path for demo data
data_path = '../../examples/example_titanic_why/'

# Read files as Pandas dataframes
local_expl = pd.read_csv(os.path.join(data_path, 'local_explainability.csv'))
local_nodes = pd.read_csv(os.path.join(data_path, 'local_graph_nodes.csv'))
qm = pd.read_csv(os.path.join(data_path, 'local_dataset_reliability.csv'))
why_elements = pd.read_csv(os.path.join(data_path, 'es', 'why_element.csv'), comment='#')
why_target = pd.read_csv(os.path.join(data_path, 'es', 'why_target.csv'), comment='#')
why_templates = pd.read_fwf(os.path.join(data_path, 'es', 'why_templates.csv'), header=None)

In [6]:
# Instantiate Why object
why = Why(language='es', local_expl=local_expl, local_nodes=local_nodes, why_elements=why_elements, why_target=why_target,
          why_templates=why_templates, quality_measure=qm, n_local_features=2, n_global_features=2, min_quality=0.7)

In [7]:
# Request a single explanation
df = why.build_why(key_column='id', key_value=24)
df

'Por ser mujer y pagar mucho por el billete, este caso ha sido clasificado como survivor, teniendo en cuenta que sobrevivieron muchas mujeres y pagaron mucho por el billete.'

In [8]:
df = why.build_why(key_column='id')
df

,id,reason
0,5,No es posible ofrecer una explicación para este caso.
1,24,"La clasificación de este caso como survivor se debe a ser mujer y pagar mucho por el billete, ya que sobrevivieron muchas mujeres y pagaron mucho por el billete."
2,50,"Por viajar en 1ª clase y ser mujer, este caso ha sido clasificado como survivor, puesto que muchos viajaban en 1ª clase y sobrevivieron muchas mujeres."
3,62,"Por embarcar en un pueblo de clase baja y ser hombre, este caso ha sido clasificado como no_survivor, teniendo en cuenta que muchos embarcaron en southampton y han muerto muchos hombres."
4,78,"La clasificación de este caso como survivor se debe a viajar en 1ª clase y ser mujer, ya que muchos viajaban en 1ª clase y sobrevivieron muchas mujeres."
...,...,...
95,1260,"Como sobrevivieron muchas mujeres y pagaron un billete de coste intermedio, y este caso se caracteriza por ser mujer y pagar un billete de coste intermedio, ha sido clasificado como survivor."
96,1269,No es posible ofrecer una explicación para este caso.
97,1283,"Por ser hombre y pagar poco por el billete, este caso ha sido clasificado como no_survivor, puesto que han muerto muchos hombres y pagaron poco por el billete."
98,1291,"Como han muerto muchos hombres y pagaron un billete de coste intermedio, y este caso se caracteriza por ser hombre y pagar un billete de coste intermedio, ha sido clasificado como no_survivor."


In [9]:
df.to_csv(os.path.join(data_path, 'reason_why_2_features.csv'), index=False, line_terminator='\n', quoting=csv.QUOTE_ALL)